In [19]:
#マルチターンデータセットを統合する

In [20]:
import glob
import json
import re
jsonl_path_list=glob.glob("data_multi_*/*.jsonl")

In [21]:

records=[]
for jsonl_path in jsonl_path_list:
    with open(jsonl_path,"r") as f:
        for line in f:
            records.append(json.loads(line))

In [22]:
jsonl_path

'data_multi_paraph/completion_records20240521120046.jsonl'

In [23]:

#きれいにする
remove_words=[
"User:",
"Assistant:",
"ユーザー：",
"アシスタント：",
"ユーザー:",
"アシスタント:",
]
def clean_chat(txt):
    for word in remove_words:
        if txt.startswith(word):
            txt=txt[len(word):]
    txt=txt.strip()
    if txt[0]=="「" and txt[-1]=="」":
        txt=txt[1:-1]


    return txt

In [24]:
txt="「アートの世界において、巨匠と呼ばれる人物は誰がありますか？」"
clean_chat(txt)

'アートの世界において、巨匠と呼ばれる人物は誰がありますか？'

In [25]:
def parse_full_auto_dialogue(txt):
    if "### 指示1:" in txt and "### 応答1:" and "### 指示2:" in txt and "### 応答2:":
        # 正規表現で「指示」と「応答」を抽出
        pattern = r"### 指示(\d+):(.*?)\n### 応答\1:(.*?)(?=\n### 指示|$)"
        matches = re.findall(pattern, txt, re.DOTALL)

        # 辞書に変換
        dialogue_dict = {}
        add_flag=True
        for match in matches:
            idx = int(match[0])
            if idx>=3:
                continue
            user_text = match[1].strip()
            assistant_text = match[2].strip()
            if len(user_text)<3:
                add_flag=False
                break
            if assistant_text=="":
                add_flag=False
                break

            if user_text.find("以下の情報を元に、UserとAssistantのやりとりを")>=0:
                add_flag=False
                break

            dialogue_dict[f"q{idx}"] = clean_chat(user_text)
            dialogue_dict[f"a{idx}"] = clean_chat(assistant_text)

        #最後にチェック
        if "q1" in dialogue_dict and "q2" in dialogue_dict and "a1" in dialogue_dict and "a2" in dialogue_dict:
            return dialogue_dict,add_flag
    
    return {},False

In [26]:
record=records[1]
dialogues=[]

invalid_records=[]
done_records=[]
for record in records:

    #自動生成のdialogue
    if "autogen_text" in record:
        txt=record["autogen_text"].strip()


    if txt in done_records:
        continue

    #質問を与える場合
    elif "question" in record and "response" in record:
        txt=record["response"].strip()
    else:
        invalid_records.append(record)
        #print("invalid record:",record)
        #raise ValueError(record)

    dialogue_dict,add_flag=parse_full_auto_dialogue(txt)
    dialogue_dict["database"]=record["database"]
    if add_flag:
        dialogues.append(dialogue_dict)
        done_records.append(txt)

In [27]:
dialogues[0]

{'q1': '「を無視できない」で終わる文章にしてください\n原子爆弾開発競争で、アメリカは黒鉛、ドイツは重水を中性子減速剤に選んだと聞きました。黒鉛に含まれるホウ素の不純物問題についてドイツは気づいていなかったからです。これについてもう少し詳しく教えていただけますか?',
 'a1': 'ホウ素の不純物問題を無視できないのは、その不純物が中性子減速剤として黒鉛の効果を低下させるためです。この不純物がホウ素から排除されずに残っていると、中性子減速過程が遅くなり、核反応を引き起こすことが困難になります。',
 'q2': '簡潔に説明を書いてください（25文字以内）',
 'a2': '不純物ホウ素低下反応効果',
 'database': 'cyberagent/chatbot-arena-ja-calm2-7b-chat-experimental'}

In [28]:
import datasets
import pandas as pd
from huggingface_hub import HfApi, logging
df=pd.DataFrame(dialogues)
df=df.reindex()
parquet_path="data_multi_paraph/1.parquet"
df.to_parquet(parquet_path)
jsonl_path="data/multi_paraph.jsonl"
with open(jsonl_path,"w") as f:
    for dialogue in dialogues:
        f.write(json.dumps(dialogue,ensure_ascii=False)+"\n")

hf = HfApi()
logging.set_verbosity_debug()
hf.upload_file(#path_or_fileobj=parquet_path,
                path_or_fileobj=jsonl_path,
                path_in_repo=f"1_para.jsonl",
                repo_id="kanhatakeyama/AutoMultiTurnByMixtral8x22b", repo_type="dataset")

About to commit to the hub: 1 addition(s), 0 copie(s) and 0 deletion(s).
Request ef0b0ac9-aa55-4033-a3ba-9cf56dfcf2bc: POST https://huggingface.co/api/datasets/kanhatakeyama/AutoMultiTurnByMixtral8x22b/preupload/main (authenticated: True)
No LFS files to upload.
Request ea0bee34-be15-463b-a693-ead7d3ec84ff: POST https://huggingface.co/api/datasets/kanhatakeyama/AutoMultiTurnByMixtral8x22b/commit/main (authenticated: True)


CommitInfo(commit_url='https://huggingface.co/datasets/kanhatakeyama/AutoMultiTurnByMixtral8x22b/commit/9ddd490df0a9cfc5a72cae7e17ca679b112e8f21', commit_message='Upload 1_para.jsonl with huggingface_hub', commit_description='', oid='9ddd490df0a9cfc5a72cae7e17ca679b112e8f21', pr_url=None, pr_revision=None, pr_num=None)

In [29]:

df.reindex()

,q1,a1,q2,a2,database
0,「を無視できない」で終わる文章にしてください\n原子爆弾開発競争で、アメリカは黒鉛、ドイツは...,ホウ素の不純物問題を無視できないのは、その不純物が中性子減速剤として黒鉛の効果を低下させるた...,簡潔に説明を書いてください（25文字以内）,不純物ホウ素低下反応効果,cyberagent/chatbot-arena-ja-calm2-7b-chat-expe...
1,「探検家」になりきって説明してください\nウェブページで最小限の労力で日付範囲の選択を可能に...,探検家として、私は次のような手法をお勧めします。日付選択を簡略化するために、日付ピッカーを使...,「もしくは」で始まる文章にしてください,もしくは、日付範囲の選択をより簡略化するために、スライダーを使用することができます。ユーザー...,cyberagent/chatbot-arena-ja-calm2-7b-chat-expe...
2,「石」になりきって説明してください\n私は石となって話します。石は地球上で最も多く見られる物...,これらを考慮した上で私の意見としては、シドニーやメルボルンがお勧めできそうです。,「要するに」で始まる文章にしてください,要するに、オーストラリアの一番良い都市を選ぶ際は、観光地や学校・大学、生活環境などを考慮する...,kunishou/databricks-dolly-15k-ja
3,"シンプルに表現してみてください（15文字以内）\nゲームの難易度レベル: レインワールド, ...",- レインワールド: 中級\n- カップヘッド: 上級\n- M&M:ダークメサイア: 中級...,「全体的に見て」で始まる文章にしてください,全体的にみて、これらのゲームの難易度は下級から上級まで多岐にわたります。下級のスーパーミート...,cyberagent/chatbot-arena-ja-calm2-7b-chat-expe...
4,「とは異なり」で始まる文章にしてください\nピーチとクッパはどちらのゲームのキャラクターですか?,とは異なり、ピーチはマリオシリーズの主要人物であるプリンセスであり、クッパは同じシリーズの最...,重要な点を一文で述べてください,ピーチとクッパは両方ともマリオシリーズに属し、それぞれ主人公と最大の悪役として知られています。,cyberagent/chatbot-arena-ja-calm2-7b-chat-expe...
...,...,...,...,...,...
6437,短い説明文を書ってください（45文字以内）\n怒ったバナナの詩を作って。,了解しました、怒ったバナナについて面白い詩を考えます。,「を考慮する」で終わる文章にしてください,了解しました、怒ったバナナの詩を作成する際、読者の好みと喜びが高まるように心掛けます。,llm-jp/oasst2-33k-ja
6438,「が考えられる」で終わる文章にしてください\n史上最高のサッカー選手とは？,多くの人たちによって、プレースティルや成果面から、ポルトガルのサッカー選手であるクリスチャヌ...,「テレビ番組」になりきって説明してください,はじめまして！今回の番組では、毎週お届けする「世界のスター選手を探る」テレビ番組となります。...,llm-jp/oasst2-33k-ja
6439,簡潔に説明してみてください（30文字以内）\n中国料理の調味料リストを教える。,中国料理でよく使われる調味料としては、生姜、ニンニク、醤油、砂糖、米酢、麻油、白胡椒粉、青ネ...,一言で要約してみてください,中国料理の調味料リスト。,kunishou/databricks-dolly-15k-ja
6440,「空」になりきって説明してください\n2000年代から2010年代前半のポップミュージック制...,2000年代と2010年代前半のポップミュージックは、多くの人が耳にする主流的な音楽でした。...,「常識的に言って」で始まる文章にしてください,常識的に言って、2000年代と2010年代前半のポップミュージックは、音楽的な多様性が減少し...,cyberagent/chatbot-arena-ja-calm2-7b-chat-expe...
